In [1]:
'''
This note is about some traditional pde algorithm i want to implement.
1.Finite difference
2.Finite volume
3.Finite element
About the pde,the laplace equation used for example.
    Lap u = -f, in Domain
    u = g = 1 on partial domain
    
    f = 2(2pi)**2sin(2pix)sin(2piy)
    u = sin(spix)sin(spiy) + 1
'''
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import torch
from scipy import sparse
from scipy.sparse.linalg import spsolve
# 1.We first declare a zero matrix perpareing for generate the stiff matrix
f = lambda x,y: 8*np.pi**2*np.sin(2*np.pi*x)*np.sin(2*np.pi*y) 
u = lambda x,y: np.sin(2*np.pi*x)*np.sin(2*np.pi*y) 

def mean_L2(pre, ans):
    pre = pre.flatten()
    ans = ans.flatten()
    diff = (pre - ans)**2
    diff = np.sqrt(diff.sum())
    return diff/len(pre)

def np2torch(A):
    values = A.data
    indices = np.vstack((A.row, A.col))
    
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = A.shape
    return torch.sparse.FloatTensor(i, v, torch.Size(shape)).to(torch.float32)

In [ ]:
# We need to know the size of this matrix.So we compute that firstly.
left = 0
right = 1
bottom = 0
top = 1

# Here we only consider the uniform mesh
dx = 0.1
dy = 0.1
Nx = round((right - left)/dx) + 1
Ny = round((top - bottom)/dy) + 1
x = np.linspace(left, right, Nx)
y = np.linspace(bottom, top, Ny)
xx, yy = np.meshgrid(x, y)
# we are facing a mesh with size of Nx * Ny 
# That means we need a matrix A with size of (Nx * Ny)**2
N = Nx * Ny
A = np.zeros((N, N))
b = np.zeros(N)
# A = sparse.lil_matrix((N, N))
# Assemble the stiffness matrix 
for i in range(Nx):
    for j in range(Ny):
        idx = i * Ny + j 
        if i == 0 or j == 0 or i==(Nx-1) or j==(Ny-1):
            A[idx, idx] = 1
            b[idx] = 0
        else:
            A[idx, idx] = -(2/dx**2 + 2/dy**2)
            A[idx, idx-1] = 1/dx**2
            A[idx, idx+1] = 1/dx**2
            A[idx, (i-1)*Ny + j] = 1/dy**2
            A[idx, (i+1)*Ny + j] = 1/dy**2            
            
            b[idx] = -f(x[i], y[j])
pre = np.linalg.solve(A, b)
ans = u(xx, yy)
print(mean_L2(pre, ans))

In [ ]:
def finite_diff_solver(f, dx, dy, left, right, bottom, top):
    Nx = round((right - left)/dx) + 1
    Ny = round((top - bottom)/dy) + 1
    N = Nx * Ny
    A = sparse.lil_matrix((N, N))
    b = np.zeros(N)
    
    for i in range(Nx):
        for j in range(Ny):
            idx = i * Ny + j
            if i == 0 or j == 0 or i==(Nx-1) or j==(Ny-1):
                A[idx, idx] = 1
                b[idx] = 0
            else:
                A[idx, idx] += (2/dx**2 + 2/dy**2)
                A[idx, idx-1] = -1/dy**2
                A[idx, idx+1] = -1/dy**2
                A[idx, (i-1)*Ny + j] = -1/dx**2
                A[idx, (i+1)*Ny + j] = -1/dx**2            

                x = left + j * dx
                y = bottom + i * dy
                b[idx] = f(x, y)
    A = A.tocsr()
    pre = spsolve(A, b)
    return pre

pre = finite_diff_solver(f, dx, dy, left, right, bottom, top)
print(mean_L2(pre, ans))

In [23]:
def assmble(a, h, f, name_A, name_b):
    n = round(a/h) + 1
    N = n**2
    A = sparse.lil_matrix((N, N))
    b = np.zeros(N)
    
    for i in range(n):
        for j in range(n):
            idx = i * n + j
            if i==0 or j==0 or i==(n-1) or j==(n-1):
                A[idx, idx] = 1
                b[idx] = 0
            else:
                A[idx, idx] = -4/h**2
                A[idx, idx-1] = 1/h**2
                A[idx, idx+1] = 1/h**2
                A[idx, (i-1)*n + j] = 1/h**2
                A[idx, (i+1)*n + j] = 1/h**2            

                b[idx] = -f(i*h, j*h)
    A = A.tocoo()    
    sparse.save_npz(name_A, A)
    np.save(name_b, b)
    return True
    
assmble(1, 0.002, f, 'A', 'b')
# print(mean_L2(pre, ans))

True